In [17]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix


data = pd.read_csv('../data/processed/clean_dataset_.csv')
components = pd.read_csv('../data/processed/principalComponents_.csv')

X = components
y = data['Fire Alarm']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)



def evaluer_modele(y_test, y_pred, y_pred_proba):
    """
    Calcule et affiche les principales métriques d'évaluation d'un modèle de classification.

    Args:
        y_test (array-like): Les vraies étiquettes.
        y_pred (array-like): Les étiquettes prédites par le modèle.
        y_pred_proba (array-like): Les probabilités prédites pour la classe positive.
    """

    # Calcul des métriques
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    roc_auc = roc_auc_score(y_test, y_pred_proba)
    conf_matrix = confusion_matrix(y_test, y_pred)

    # Création du DataFrame pour afficher les résultats
    data = {'Métrique': ['Accuracy', 'Precision', 'Recall', 'F1-score', 'AUC', 'Matrice de confusion'],
            'Valeur': [accuracy, precision, recall, f1, roc_auc, conf_matrix]}
    df = pd.DataFrame(data)

    print(df)


#Réseau de neurone
#regression linéaire
#XGBoost


In [18]:
#---------------------------------   Random Forest  -------------------------------------------------------
# Créer un modèle de forêt aléatoire
model1 = RandomForestClassifier(n_estimators=100, random_state=42)

# Entraîner le modèle
model1.fit(X_train, y_train)

# Faire des prédictions   
y_pred = model1.predict(X_test)

# Obtenir les probabilités prédites
y_pred_proba = model1.predict_proba(X_test)[:, 1] 


evaluer_modele(y_test, y_pred, y_pred_proba)

               Métrique                  Valeur
0              Accuracy                0.999655
1             Precision                     1.0
2                Recall                0.999511
3              F1-score                0.999756
4                   AUC                0.999732
5  Matrice de confusion  [[2552, 0], [3, 6138]]


In [22]:
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Diviser les données en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Créer un modèle XGBoost
model = XGBClassifier()

# Entraîner le modèle
model.fit(X_train, y_train)

# Faire des prédictions
y_pred2 = model.predict(X_test)

# Obtenir les probabilités prédites
y_pred_proba2 = model1.predict_proba(X_test)[:, 1] 


evaluer_modele(y_test, y_pred2, y_pred_proba2)

               Métrique                  Valeur
0              Accuracy                0.999655
1             Precision                     1.0
2                Recall                0.999511
3              F1-score                0.999756
4                   AUC                0.999732
5  Matrice de confusion  [[2552, 0], [3, 6138]]


In [24]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error



# Diviser les données en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Créer un modèle de régression linéaire
model = LinearRegression()

# Entraîner le modèle
model.fit(X_train, y_train)

# Faire des prédictions
y_pred3 = model.predict(X_test)

# Obtenir les probabilités prédites
y_pred_proba2 = model1.predict_proba(X_test)[:, 1] 


evaluer_modele(y_test, y_pred2, y_pred_proba2)

# Évaluer le modèle
mse = mean_squared_error(y_test, y_pred)
print("Mean Squared Error:", mse)

               Métrique                  Valeur
0              Accuracy                0.999655
1             Precision                     1.0
2                Recall                0.999511
3              F1-score                0.999756
4                   AUC                0.999732
5  Matrice de confusion  [[2552, 0], [3, 6138]]
Mean Squared Error: 0.00034510525710341653
